In [16]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [17]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [18]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [19]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [20]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [21]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [22]:
df['loc']

1     (8955 Collins Ave APT 201, Surfside, FL 33154,...
2     (18975 Collins Ave #1802, Sunny Isles Beach, F...
3     (10261 E Bay Harbor Dr #1201, Bay Harbor Islan...
4     (3131 NE 7th Ave #4301, Miami, FL 33137, USA, ...
5     (19333 Collins Ave APT 1207, Sunny Isles Beach...
6     (17021 Collins Ave #3903, Sunny Isles Beach, F...
7     (2020 N Bayshore Dr APT 3401, Miami, FL 33137,...
8     (6515 Collins Ave APT 1908, Miami Beach, FL 33...
9     (3131 NE 7th Ave #2303, Miami, FL 33137, USA, ...
10    (10201 E Bay Harbor Dr Apt 306, Bay Harbor Isl...
Name: loc, dtype: object

In [23]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [24]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Arte Surfside\r8955 Collins Ave 201\rMiami Beach,1/24/2023,671,"$17,000,000.00",8463,"$2,437.28",Dina Goldentayer,Douglas Elliman,Isabel Castro,Societe Real Estate,Arte Surfside 8955 Collins Ave 201 Miami Beach,Arte Surfside,8955 Collins Ave 201 Miami Beach,"(8955 Collins Ave APT 201, Surfside, FL 33154,...","(25.8758278, -80.1210758, 0.0)",25.875828,-80.121076,0.0
2,Residences by Armani Casa\r18975 Collins Ave 1...,1/24/2023,235,"$4,100,000.00",4062,"$1,290.93",Mariana Niro,Douglas Elliman,Dina Goldentayer,Compass Florida LLC,Residences by Armani Casa 18975 Collins Ave 18...,Residences by Armani Casa,18975 Collins Ave 1802 Sunny Isles Beach,"(18975 Collins Ave #1802, Sunny Isles Beach, F...","(25.9521284, -80.11970810000001, 0.0)",25.952128,-80.119708,0.0
3,Le Nautique at Haulover\r10261 E Bay Harbor Dr...,1/23/2023,74,"$2,800,000.00",3200,$875.00,Daniel Tzinker,The Agency Florida LLC,Non Member Listing Agent,The Agency Florida LLC,Le Nautique at Haulover 10261 E Bay Harbor Dr ...,Le Nautique at Haulover,10261 E Bay Harbor Dr 1201 Bay Harbor Islands,"(10261 E Bay Harbor Dr #1201, Bay Harbor Islan...","(25.8932241, -80.13125509999999, 0.0)",25.893224,-80.131255,0.0
4,One Paraiso\r3131 NE 7th Ave 4301\rMiami,1/25/2023,184,"$1,700,000.00",1634,"$1,040.39",Daniela Pellicciotti,Condoideas Realty Group LLC,Fabrizio Pellicciotti,Societe Real Estate,One Paraiso 3131 NE 7th Ave 4301 Miami,One Paraiso,3131 NE 7th Ave 4301 Miami,"(3131 NE 7th Ave #4301, Miami, FL 33137, USA, ...","(25.8072459, -80.18560169999999, 0.0)",25.807246,-80.185602,0.0
5,Ocean One Condo\r19333 Collins Ave 1207\rSunny...,1/24/2023,1,"$1,632,500.00",1756,$929.67,Joanna Jimenez,Compass Florida LLC,Krisztian Nagy,Compass Florida LLC,Ocean One Condo 19333 Collins Ave 1207 Sunny I...,Ocean One Condo,19333 Collins Ave 1207 Sunny Isles Beach,"(19333 Collins Ave APT 1207, Sunny Isles Beach...","(25.955682, -80.11954759999999, 0.0)",25.955682,-80.119548,0.0
6,Ocean Four Condominium\r17021 Collins Ave 3903...,1/26/2023,40,"$1,470,000.00",1644,$894.16,Valentina Wish,Beachfront Realty Inc,Natalia Figueroa,"Florida Best Realty, Inc.",Ocean Four Condominium 17021 Collins Ave 3903 ...,Ocean Four Condominium,17021 Collins Ave 3903 Sunny Isles Beach,"(17021 Collins Ave #3903, Sunny Isles Beach, F...","(25.9341829, -80.1209647, 0.0)",25.934183,-80.120965,0.0
7,Paramount Bay\r2020 N Bayshore Dr 3401\rMiami,1/23/2023,68,"$1,450,000.00",1588,$913.10,Michael Mann,One World Properties,Maria Kunigonis,Miami Realty Solution Group,Paramount Bay 2020 N Bayshore Dr 3401 Miami,Paramount Bay,2020 N Bayshore Dr 3401 Miami,"(2020 N Bayshore Dr APT 3401, Miami, FL 33137,...","(25.7966352, -80.18792409999999, 0.0)",25.796635,-80.187924,0.0
8,Bel-Aire on the OCean\r6515 Collins Ave PH-190...,1/23/2023,34,"$1,395,000.00",1097,"$1,271.65",Glen Janney,Cervera Real Estate Inc.,Gustavo Blachman,The Keyes Company,Bel-Aire on the OCean 6515 Collins Ave PH-1908...,Bel-Aire on the OCean,6515 Collins Ave PH-1908 Miami Beach,"(6515 Collins Ave APT 1908, Miami Beach, FL 33...","(25.8483641, -80.1196763, 0.0)",25.848364,-80.119676,0.0
9,One Paraiso\r3131 NE 7th Ave 2303\rMiami,1/23/2023,41,"$1,350,000.00",1526,$884.67,Adam Redolfi,Barnes Int'l Realty LLC,Tatiana Costa-Martinez,Compass Florida LLC,One Paraiso 3131 NE 7th Ave 2303 Miami,One Paraiso,3131 NE 7th Ave 2303 Miami,"(3131 NE 7th Ave #2303, Miami, FL 33137, USA, ...","(25.8072459, -80.18560169999999, 0.0)",25.807246,-80.185602,0.0
10,0Sereno\r10201 E Bay Harbor Dr 306\rBay Harbor...,1/23/2023,4,"$1,300,000.00",1067,"$1,218.37",Batsheva Rivkin,One Sotheby's Int'l Realty,Todd Nordstrom,One Sotheby's Int'l Realty,0Sereno 10201 E Bay Harbor Dr 306 Bay Harbor I...,,0Sereno 10201 E Bay Harbor Dr 306 Bay Harbor I...,"(10201 E Bay Harbor Dr Apt 306, Bay Harbor Isl..."

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [25]:
df.at[8,'building_name']=('Bel-Aire on the Ocean')

df.at[10,'building_name']=('Sereno')
df.at[10,'address_only']=('10201 E Bay Harbo Drive')

In [26]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [27]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 22nd - January 28th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[2]}{bold_end}{new_line}Address: {bold_start}{info[3]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [28]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [30]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_013022
